# Fancy Decorators

In the second part of this tutorial, we explore more advanced
features including:

- Add **decoractors to classes**
- Add **several decorators** to one function
- Create decorators with **arguments**
- Create decorators that can **optionally** take arguments
- Define **stateful** decorators
- **Define classes** that act as decorators

## Decorating Classes

One can use decorators on classes in two different ways

- **Decorate** the **methods of a class**
- Decorate the **whole** class

Some decorations for the methods of a class are even built-ins in
Python, including:

- `@classmethod`
- `@staticmethod`
- `@property`


For example, we can decorate methods using the `@trace` and `@timer`
decorators defined earlier.

In [ ]:
from decorators import trace, timer

class TimeWaster:
    @trace
    def __init__(self, max_num):
        self.max_num = max_num

    @timer
    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([number ** 2 for number in range(self.max_num)])

Using this class, one can see the effect of the decorators.

In [ ]:
tw = TimeWaster(1000)

In [ ]:
tw.waste_time(999)

The other way to use decorators is to **decorate the whole class**.

This decoration of the entire class is done, for example, in the
`dataclasses` module.

In [ ]:
from dataclasses import dataclass

@dataclass
class PlayingCard:
    rank: str
    suit: str

The meaning of the syntax is similar to that of a function decorator.
For exaple, one could have decorated the `PlayingCard` class by the
expression, `PlayingCard = dataclass(PlayingCard)`.

A common use case of class decorators is to be a simpler alternative
to some use cases of **metaclasses**. In both cases, one is changing
the definition of a class **dynamically**.

Writing a class decorator is similar to writing a function decorator.
The only difference is that the decorator will receive a **class**
and not a function as an arguments. In fact, all the decorators in
our previous notebooks **will** work as class decorators.

However, when you're using these decorators on a class method instead
of a function, their effect **might not be what you want.

For example, let's apply the `@timer` decorator to a class.

In [ ]:
@timer
class TimeWaster:
    def __init__(self, max_num):
        self.max_num = max_num

    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([i ** 2 for i in range(self.max_num)])

Decorating a class **doesn't** decorate its methods. Instead, `@timer`
only measures the time it takes to instantiate the class.

In [ ]:
tw = TimeWaster(1000)

In [ ]:
tw.waste_time(999)

Note that the output of `@timer` is only printed when the instance,
`tw`, is created. The call to `waste_time()` **is not timed**.

Later, we'll see an example of defining a proper class decorator,
namely, `@singleton`, which ensures that only one instance of
class exists at runtime.

## Nesting Decorators

One can _apply several decorators_ to a function at once by stacking
them on top of each other.

In [ ]:
from decorators import trace, do_twice

@trace
@do_twice
def greet(name):
    print(f'Hello {name}')

Remember that applying two decorators as we do previously results in

- `@trace calls @do_twice which calls `greet()`

which is the same as `trace(do_twice(greet()))

In [ ]:
greet('Yadi')

The greeting is printed twice because of `@do_twice`. However, the
output from `@trace` is only shown **once** since the call to `trace`
occurs **before** the `@do_twice` decorator.

In [ ]:
@do_twice
@trace
def greet(name):
    print(f'Hello {name}')

greet('Yadi')

Here, `@do_twice` is applied to @trace. Consequently, one can see that
both calls to `greet()` are annotated with trace information.

## Defining Decorators with Arguments

Sometimes, it is useful to _pass arguments to your decorators_. For
instance, `@do_twice` could be extended to a `@repeat(num_times)`
decorator. In this situation, the number of repetitions could be
given as an argument.

In [ ]:
from decorators import repeat

There are a few subtle things happening in the `repeat()` function:

- Defining `decorator_repeat()` as an inner function means that
  `repeat()` refers to a function object, `decorator_repeat`. Earlier,
  you used decorators like `@do_twice` **without** parentheses. Now,
  you **must** add parentheses when setting up the decorator, as in
  `@repeat()`. This is required to **add arguments**.
- The `num_times` argument is seemingly not used in `repeat()` itself.
  But by passing `num_times` to `repeat()`, a **closure** is created
  where the value of `num_times` is stored until `wrapper_repeat()`
  uses it later.

Here are our results:

In [ ]:
@repeat(num_times=4)
def greet(name):
    print('Hello', name)

greet('World)')

## Creating Decorators with Optional Arguments

With a little bit of care, you can also define _decorators that can
be used both with and without arguments.

When a decorator uses arguments, you need to add an extra outer function.
The challenge now is for your code to figure out if you've called the
decorator with or without arguments. This means that the decorator
arguments must all be specified by keyword. You can enforce this with the
special asterisk (`*`) syntax, which means that **all the following
parameters are keyword-only**.

For example, after defining the inner decorator, for example,
`decorator_name()`, return `decorator_name` if `_func` is `None`, return
`decorator_name` (the function **without** any captured `_func`). If
`_func` is **not** `None`, return `decorator_name(_func)`; that is,
the `decorator_name` function with the `_func` argument set.


Let's now apply `repeat` with and without arugements.

In [ ]:
@repeat
def say_whee():
    print('Whee!')

say_whee()

In [ ]:
@repeat(num_times=3)
def greet(name):
    print(f'Hello {name}')

greet('Penny')

## Tracking State in Decorators

Sometimes, it's useful to have a decorator that can **keep track
of state**. As an example, we'll create a decorator that counts
the number of times a function is called.

**Note**: In the beginning of this guide, you learned about pure
functions returning a value based on given arguments. Stateful
decorators are quite the opposite, where the return value will
depend on the current state, as well as the given arguments.

In the next section, you'll see how to use classes to keep state.
But in simple cases, you can also get away with using
**function attributes**.

Here's the effect of our `count_calls` decorator with side-effects.

In [ ]:
from decorators import count_calls

@count_calls
def say_whee():
    print('Whee!')

In [ ]:
say_whee()

say_whee()

say_whee.num_calls

## Using Classes as Decorators

The typical way to maintain state in Python is by **using classes**.
In this section, you'll see how to rewrite the `@count_calls` example
from the previous section to _use a class as a decorator_.

Recall that the decorator syntax `@decorator` is just a quicker way of
saying `func = decorator(func)`. Consequently, if a decorator is a
**class**, it needs to take `func` as an argument in its `__init__()`
initializer. Furthermore, the class instance needs to be **callable**
so that it can stand in for a _decorated function_.

**Note**: Up until now, all the decorators that you've seen have been
defined as functions. This is how one most often creates decorators.
However, one can use **any callable expression** as a decorator.

For a **class instance** to be callable, one implements the special method, `__call__()`.

In [ ]:
class Counter:
    def __init__(self, start=0):
        self.count = start

    def __call__(self):
        self.count += 1
        print(f'Current count is {self.count}')

The `__call__()` method is executed each time you call an
**instance** of the class.

In [ ]:
counter = Counter()
counter()

In [ ]:
counter()

In [ ]:
counter.count

Each time one calls `counter()`, the **state** changes; that is,
count` increases.

Consequently, a typical implementation of a decorator class
should implement both `__init__()` and `__call__()`.

(See the `CountCalls` class in file, `decorators.py`.)

The imported `CountCalls` decorator works the same as the
`Counter()` function.

In [ ]:
from decorators import CountCalls

@CountCalls
def say_whee():
    print('Whee!')

In [ ]:
say_whee()

In [ ]:
say_whee()

In [ ]:
say_whee.num_calls

Each call to `say_whee()` is counted and noted.